# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../WeatherPy/world_cities.csv")
vacation_df.head()

,City_ID,City,Country,Lat,Long,Temp (F),Humidity,Cloudiness,Wind Speed,Date
0,1,methoni,GR,36.82,21.70,82.40,51,40,13.87,1593087971
1,2,hermanus,ZA,-34.42,19.23,70.00,54,39,5.99,1593087972
2,3,jamestown,US,42.10,-79.24,60.80,77,20,5.82,1593087973
3,4,upernavik,GL,72.79,-56.15,35.87,91,100,3.18,1593087974
4,5,albany,US,42.60,-73.97,70.00,50,5,3.60,1593087888


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Long"]].astype(float)

# Convert humidity to float and store
humidity = vacation_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(center=(10,40),zoom_level=3)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = vacation_df[(vacation_df["Temp (F)"] >= 75) & (vacation_df["Temp (F)"] <= 80) &\
                 (vacation_df["Wind Speed"] < 10) & (vacation_df["Cloudiness"] == 0)]

hotel_df = hotel_df.dropna()
hotel_df

,City_ID,City,Country,Lat,Long,Temp (F),Humidity,Cloudiness,Wind Speed,Date
92,93,east london,ZA,-33.02,27.91,78.80,19,0,5.82,1593088069
244,245,starodub,RU,52.58,32.76,75.02,58,0,5.88,1593088249
289,290,jimo,CN,36.39,120.46,77.00,78,0,8.95,1593088300
317,318,caravelas,BR,-17.71,-39.25,79.25,61,0,5.82,1593088332
351,352,lowestoft,GB,52.48,1.75,77.00,69,0,8.99,1593088370
488,489,zaoyang,CN,32.13,112.75,79.92,77,0,7.29,1593088517
494,495,baoying,CN,33.23,119.31,77.36,67,0,4.29,1593088523
527,528,plettenberg bay,ZA,-34.05,23.37,77.18,29,0,7.83,1593088487
531,532,guarapari,BR,-20.67,-40.50,75.99,66,0,8.88,1593088394
566,567,lerici,IT,44.08,9.91,78.80,57,0,8.05,1593088599


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Country,Lat,Long,Temp (F),Humidity,Cloudiness,Wind Speed,Date,Hotel Name
92,93,east london,ZA,-33.02,27.91,78.80,19,0,5.82,1593088069,
244,245,starodub,RU,52.58,32.76,75.02,58,0,5.88,1593088249,
289,290,jimo,CN,36.39,120.46,77.00,78,0,8.95,1593088300,
317,318,caravelas,BR,-17.71,-39.25,79.25,61,0,5.82,1593088332,
351,352,lowestoft,GB,52.48,1.75,77.00,69,0,8.99,1593088370,
488,489,zaoyang,CN,32.13,112.75,79.92,77,0,7.29,1593088517,
494,495,baoying,CN,33.23,119.31,77.36,67,0,4.29,1593088523,
527,528,plettenberg bay,ZA,-34.05,23.37,77.18,29,0,7.83,1593088487,
531,532,guarapari,BR,-20.67,-40.50,75.99,66,0,8.88,1593088394,
566,567,lerici,IT,44.08,9.91,78.80,57,0,8.05,1593088599,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_list = requests.get(base_url, params=params)
    
    # convert to json
    hotel_list = hotel_list.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_list["results"][0]["name"]
       
    except:
        print("Missing field/result... skipping.")

In [8]:
hotel_df

,City_ID,City,Country,Lat,Long,Temp (F),Humidity,Cloudiness,Wind Speed,Date,Hotel Name
92,93,east london,ZA,-33.02,27.91,78.80,19,0,5.82,1593088069,Blue Lagoon Hotel
244,245,starodub,RU,52.58,32.76,75.02,58,0,5.88,1593088249,Apartment Kvartal on Gagarina
289,290,jimo,CN,36.39,120.46,77.00,78,0,8.95,1593088300,DoubleTree by Hilton Hotel Qingdao-Jimo
317,318,caravelas,BR,-17.71,-39.25,79.25,61,0,5.82,1593088332,Mar Doce de Abrolhos
351,352,lowestoft,GB,52.48,1.75,77.00,69,0,8.99,1593088370,Ivy House Country Hotel
488,489,zaoyang,CN,32.13,112.75,79.92,77,0,7.29,1593088517,Hancheng Hotel
494,495,baoying,CN,33.23,119.31,77.36,67,0,4.29,1593088523,South Beauty Hotel
527,528,plettenberg bay,ZA,-34.05,23.37,77.18,29,0,7.83,1593088487,The Plettenberg Hotel
531,532,guarapari,BR,-20.67,-40.50,75.99,66,0,8.88,1593088394,Hotel Porto do Sol
566,567,lerici,IT,44.08,9.91,78.80,57,0,8.05,1593088599,Piccolo Hotel del Lido


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))